In [1]:
import cv2
import torch
import torchvision

In [2]:
device = torch.device('cuda')

In [3]:
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('trt_collision_model_sala230_50.pth'))

<All keys matched successfully>

In [4]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_device=0, capture_fps = 30)

image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# display(widgets.VBox([widgets.HBox([image, blocked_slider]), speed_slider]))

In [6]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

In [7]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model_trt(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        car.throttle = -speed_slider.value
    else:
        car.throttle = 0.0
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to initialize

In [8]:
camera.running = True
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

In [9]:
display(widgets.VBox([widgets.HBox([image, blocked_slider]), speed_slider]))

In [ ]:
#STOP
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

car.throttle = 0.0